In [15]:
import requests
import time
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup


In [62]:
r = requests.get('https://www.pro-football-reference.com/players/A/AbduKa00.htm')
time.sleep(.04)
soup = BeautifulSoup(r.content, "lxml")

In [63]:
rush_rec_table_list = soup.find_all("div", {"id": "div_rushing_and_receiving"})

if len(rush_rec_table_list) == 0:
    # either make a bunch of 0s or TBD
    pass
else:
# there should only be one       
# now get all of the shit
    rush_rec_table = rush_rec_table_list[0]
    
    #for row in rush_rec_table.find_all("tr"):
    #    print(row)





In [66]:
start_yr = 1996

#df = pd.DataFrame()

for year_num in range(4):
    this_year_stats = rush_rec_table.find("tr", {"id": "rushing_and_receiving."+str(start_yr+year_num)})
    
    if not this_year_stats:
        # placeholder
        print(start_yr+year_num)
        continue
        
    
    for datum in this_year_stats.find_all("td"):
        shorter = str(datum).split("data-stat")[1]
        stat_name = shorter.split('"')[1]
        # TODO: add ProBowl in year x and All Pro in year x
        
        try:
            if stat_name =="team":
                data = shorter.split('title="')[1].split('"')[0]
            else:
                data = shorter.split(">")[1].split("<")[0]

            if stat_name =="team" or stat_name == "pos":
                df[stat_name+"_"+str(year_num)] = [data.upper()]
            else:
                if stat_name == "catch_pct":
                    data = data.split("%")[0]

                df[stat_name+"_"+str(year_num)] = [float(data)]
        except:
            df[stat_name+"_"+str(year_num)] ="?"

In [67]:
df

,age_0,team_0,pos_0,uniform_number_0,g_0,gs_0,rush_att_0,rush_yds_0,rush_td_0,rush_long_0,...,rec_yds_per_rec_3,rec_td_3,rec_long_3,rec_per_g_3,rec_yds_per_g_3,catch_pct_3,yds_from_scrimmage_3,rush_receive_td_3,fumbles_3,av_3
0,22.0,MIAMI DOLPHINS,RB,33.0,16.0,14.0,307.0,1116.0,11.0,29.0,...,4.9,1.0,21.0,1.3,6.5,77.3,529.0,2.0,0.0,4.0


In [55]:
rush_rec_table.find("tr", {"id": "rushing_and_receiving.2016"})

# Thoughts on structure:

We need to figure out what we want to do once we have this category df, we want to concat this into a larger df for a player. Once all categories are handled, we want to write this to a csv. All players only see their own df, but the write to csv function make the actual one we will use for ML

In [23]:
all_players = pd.read_csv("../data/player_data1985-2005.csv", index_col = 0)

In [24]:
all_players.head()

,HOF,age_0,age_1,age_2,age_3,all_pro_0,all_pro_1,all_pro_2,all_pro_3,all_purpose_yds_0,...,xpa_2,xpa_3,xpm_0,xpm_1,xpm_2,xpm_3,yds_from_scrimmage_0,yds_from_scrimmage_1,yds_from_scrimmage_2,yds_from_scrimmage_3
0,0,28.0,29.0,NaN,NaN,False,False,NaN,NaN,NaN,...,NaN,NaN,22.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0,22.0,23.0,24.0,25.0,False,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1255.0,1153.0,1062.0,529.0
0,0,24.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,24.0,25.0,26.0,27.0,False,False,False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23.0,84.0,66.0,0.0
0,0,23.0,24.0,NaN,NaN,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
all_players_df = all_players.replace(np.NaN, "?")

In [26]:
all_players_df.to_csv("../data/new_player_data1985-2005.csv", index=False)

In [27]:
all_players_df.shape

(3776, 386)

In [28]:
all_players_df.head()

,HOF,age_0,age_1,age_2,age_3,all_pro_0,all_pro_1,all_pro_2,all_pro_3,all_purpose_yds_0,...,xpa_2,xpa_3,xpm_0,xpm_1,xpm_2,xpm_3,yds_from_scrimmage_0,yds_from_scrimmage_1,yds_from_scrimmage_2,yds_from_scrimmage_3
0,0,28.0,29,?,?,False,False,?,?,?,...,?,?,22,15,?,?,?,?,?,?
0,0,22.0,23,24,25,False,False,False,False,?,...,?,?,?,?,?,?,1255,1153,1062,529
0,0,24.0,?,?,?,False,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
0,0,24.0,25,26,27,False,False,False,False,?,...,?,?,?,?,?,?,23,84,66,0
0,0,23.0,24,?,?,False,False,?,?,?,...,?,?,?,?,?,?,?,?,?,?


In [29]:
all_players["HOF"]

0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    1
0    0
0    0
0    0
0    0
0    0
0    0
    ..
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    1
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    0
0    1
0    0
0    0
0    0
0    0
Name: HOF, dtype: int64